Imports

In [1]:
from midrc_dataset import midrc_SIMCLR_dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from get_model import create_model
import torchvision
import torch
import os
import torch.nn as nn
import numpy as np
import cross_fold

Dataset Configuration

In [2]:
class SimCLR_Loss(nn.Module):
    def __init__(self, batch_size, temperature):
        super().__init__()
        self.batch_size = batch_size
        self.temperature = temperature

        #self.mask = self.mask_correlated_samples(batch_size)
        self.criterion = nn.CrossEntropyLoss(reduction="sum")
        self.similarity_f = nn.CosineSimilarity(dim=2)

    def mask_correlated_samples(self, batch_size):
        N = 2 * batch_size
        mask = torch.ones((N, N), dtype=bool)
        mask = mask.fill_diagonal_(0)
        
        for i in range(batch_size):
            mask[i, batch_size + i] = 0
            mask[batch_size + i, i] = 0
        return mask

    def forward(self, z_i, z_j):

        mask = self.mask_correlated_samples(z_i.shape[0])

        N = 2 * z_i.shape[0]

        z = torch.cat((z_i, z_j), dim=0)

        sim = self.similarity_f(z.unsqueeze(1), z.unsqueeze(0)) / self.temperature

        sim_i_j = torch.diag(sim, z_i.shape[0])
        sim_j_i = torch.diag(sim, -z_i.shape[0])
        
        # We have 2N samples, but with Distributed training every GPU gets N examples too, resulting in: 2xNxN
        #print(sim_i_j.shape, sim_j_i.shape)
        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(N, 1)
        #print("Positive Done")
        negative_samples = sim[mask].reshape(N, -1)
        #print("Negative Done")
        
        #SIMCLR
        labels = torch.from_numpy(np.array([0]*N)).reshape(-1).to(positive_samples.device).long() #.float()
        
        logits = torch.cat((positive_samples, negative_samples), dim=1)
        loss = self.criterion(logits, labels)
        loss /= N
        
        return loss

In [3]:
def train(epochs,model,device, train_loader, val_loader, criterion, optimizer, experiment_name, fold_number):
    metrics={}
    best_vloss = 1_000_000.
    for e in range(epochs):
        print('EPOCH {}:'.format(e + 1))
        running_loss = 0.
        last_loss = 0.
        running_train_kappa=0.0
        avg_train_kappa=0.0
        model.train(True)

        for i, data in enumerate(train_loader):
            # Every data instance is an input + label pair
            img1, img2, score1, score2 = data

            #print("img1 and img2 shape", img1.shape, img2.shape)

            # Zero your gradients for every batch!
            optimizer.zero_grad()

            # Make predictions for this batch
            output1 = model(img1.to(device))
            output2 = model(img2.to(device))

            # Compute the loss and its gradients
            loss = criterion(output1, output2) 
            loss.backward()

            # Adjust learning weights
            optimizer.step()
        
            # Gather data
            running_loss += loss
        
        avg_loss = running_loss/(i+1)
        
        running_vloss = 0.0
        running_kappa = 0.0
        model.eval()
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                img1, img2, score1, score2 = vdata
                voutput1 = model(img1.to(device))
                voutput2 = model(img2.to(device))
                vloss = criterion(output1,output2) # TODO: Create cosine sim loss
                running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        
        print('LOSS train {} valid {}'.format(avg_loss,avg_vloss))
        metrics[e] = {      
            f'Fold {fold_number} avg_train_loss': avg_loss.item(),
            f'Fold {fold_number} avg_val_loss': avg_vloss.item(),
            f'Fold {fold_number} epoch': e
        }
        #if config['wandb']==True:
        #    wandb.log(metrics[e])

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            model_path = os.path.join('experiments',experiment_name,'saved_models', 'modelsave_fold_{}.ckpt'.format(fold_number))
            torch.save(model.state_dict(), model_path)

    return metrics

In [4]:
folds = cross_fold.create_folded_datasets("../data/label_info/labels.json")

epochs = 50

batch_size = 13

root_dir = '../data/resized_224X224'

json_file = '../data/label_info/refined_patient_wise_images.json'

annotations_file = 'MIDRC mRALE Mastermind Training Annotations_2079_20230428.csv'

experiment_name = 'ResNet-50 Contrastive Pretraining Per Fold'

ex_directory = os.path.join('experiments',experiment_name)
if not os.path.exists(ex_directory):
        os.makedirs(ex_directory)

models_directory = os.path.join(ex_directory,'saved_models')
if not os.path.exists(models_directory):
        os.makedirs(models_directory)

transform = transforms.Compose([transforms.ToTensor()])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def make_contrastive_pretrained_model():
        model = torchvision.models.resnet50(weights='ResNet50_Weights.DEFAULT')
        model.conv1 = torch.nn.Conv2d(1,64, kernel_size=(7,7),stride=(2,2),padding=(3,3), bias=False)
        model.fc = torch.nn.Sequential(
                    torch.nn.Linear(2048, 512, bias=True),
                    torch.nn.Linear(512, 256, bias=True))
        model.to(device)
        return model

criterion = SimCLR_Loss(batch_size, temperature=0.5)

saved_metrics = []

for f_i,fold in enumerate(folds):
        print("FOLD: ",f_i+1)
        train_list, val_list = fold

        model = make_contrastive_pretrained_model()

        optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001)

        train_dataset = midrc_SIMCLR_dataset(root_dir, annotations_file,json_file, transform = transform, fp_list = train_list)
        val_dataset = midrc_SIMCLR_dataset(root_dir, annotations_file,json_file, transform=transform, fp_list = val_list)

        print(len(train_dataset))
        print(len(val_dataset))
        
        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True)

        #Training per fold
        metrics = train(epochs,model,device,train_loader,val_loader, criterion, optimizer,experiment_name, fold_number=f_i+1) # Folds will be started from 1 instead of 0
        saved_metrics.append(metrics)


Fold:  1  Train Indices:  2078  Val indices:  520
Fold:  2  Train Indices:  2096  Val indices:  502
Fold:  3  Train Indices:  2050  Val indices:  548
Fold:  4  Train Indices:  2072  Val indices:  526
Fold:  5  Train Indices:  2096  Val indices:  502
FOLD:  1
172
44
EPOCH 1:
LOSS train 1.8970226049423218 valid 0.22975686192512512
EPOCH 2:
LOSS train 1.506740689277649 valid 0.24054855108261108
EPOCH 3:
LOSS train 1.4740350246429443 valid 0.32486462593078613
EPOCH 4:
LOSS train 1.5043015480041504 valid 0.5651783347129822
EPOCH 5:
LOSS train 1.4851230382919312 valid 0.24748072028160095
EPOCH 6:
LOSS train 1.5221606492996216 valid 0.3497157096862793
EPOCH 7:
LOSS train 1.464235544204712 valid 0.23155587911605835
EPOCH 8:
LOSS train 1.4599196910858154 valid 0.31904396414756775
EPOCH 9:
LOSS train 1.4409003257751465 valid 0.227624773979187
EPOCH 10:
LOSS train 1.4630277156829834 valid 0.30672022700309753
EPOCH 11:
LOSS train 1.4600430727005005 valid 0.26307082176208496
EPOCH 12:
LOSS train 1.